<a href="https://colab.research.google.com/github/JashandeepKairway/Ensemble-PreTrainedDeepFakeModels/blob/main/Ensemble_PreTrainedDeepFakeModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install transformers datasets evaluate -q
!pip install timm -q  # For image models

# Step 1: Imports
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, AutoModelForImageClassification
import evaluate
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

# Step 2: Load Dataset (You can replace this with your custom dataset path or URL)
from torchvision.datasets.utils import download_and_extract_archive

# Sample Deepfake Dataset (replace with actual)
# Example: https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces
# Simulating with ImageFolder format
dataset_path = "saakshigupta/deepfake_dataset_simple"  # Upload your dataset here
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
dataset = ImageFolder(dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Step 3: Pre-trained Model List (Use Hugging Face model names)
model_names = [
    "nateraw/vit-age-classifier",  # Example only; replace with actual deepfake models
    "facebook/deit-base-distilled-patch16-224",
    "microsoft/swin-tiny-patch4-window7-224"
]

# Step 4: Evaluation Function
def evaluate_model(model_name):
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = AutoModelForImageClassification.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    y_true, y_pred = [], []

    with torch.no_grad():
        for imgs, labels in val_loader:
            inputs = processor(images=imgs, return_tensors="pt").to(model.device)
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            y_pred.extend(predictions)
            y_true.extend(labels.numpy())

    # Compute metrics
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    sensitivity = tp / (tp + fn + 1e-8)
    specificity = tn / (tn + fp + 1e-8)
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    accuracy = accuracy_score(y_true, y_pred)

    return [sensitivity, specificity, precision, recall, f1, accuracy]

# Step 5: Loop Over Models
import pandas as pd

results = []
for model_name in model_names:
    print(f"Evaluating {model_name}...")
    metrics = evaluate_model(model_name)
    results.append(metrics)

# Step 6: Convert to DataFrame and Add Ensemble Row (majority voting not shown)
df = pd.DataFrame(results, columns=["Sensitivity", "Specificity", "Precision", "Recall", "F1", "Accuracy"])
df.insert(0, "Models", [f"M{i+1}" for i in range(len(results))])
ensemble_row = ["Ensemble"] + list(np.mean(df.iloc[:, 1:], axis=0))  # simple average
df.loc[len(df)] = ensemble_row

# Step 7: Display Table
from IPython.display import display
display(df)
